# STL Plotting Demo

This notebook demonstrates STL model loading and rendering with matplotlib.

## Features
- Load STL files from MongoDB
- Test PyVista, trimesh, and numpy-stl loading
- Render with matplotlib using Poly3DCollection
- Test wireframe and filled rendering modes
- Validate triangle geometry


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Try to import PyVista
PYVISTA_AVAILABLE = False
try:
    import pyvista as pv
    PYVISTA_AVAILABLE = True
    print("✅ PyVista available")
except ImportError:
    print("⚠️ PyVista not available")

# Try to import trimesh
TRIMESH_AVAILABLE = False
try:
    import trimesh
    TRIMESH_AVAILABLE = True
    print("✅ trimesh available")
except ImportError:
    print("⚠️ trimesh not available")

# Try to import numpy-stl
NUMPY_STL_AVAILABLE = False
try:
    from stl import mesh
    NUMPY_STL_AVAILABLE = True
    print("✅ numpy-stl available")
except ImportError:
    print("⚠️ numpy-stl not available")

# MongoDB setup
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded")

# Import MongoDB and STL clients
from src.infrastructure.database.mongodb_client import MongoDBClient
from src.am_qadf.query.stl_model_client import STLModelClient

print("\n✅ Setup complete!")


✅ PyVista available
✅ trimesh available
⚠️ numpy-stl not available
✅ Environment variables loaded

✅ Setup complete!


In [2]:
# Connect to MongoDB and get STL file path
mongo_client = MongoDBClient()
mongo_client.connect()

stl_client = STLModelClient(mongo_client=mongo_client)

# List available models
from src.am_qadf.voxel_domain.voxel_storage import VoxelGridStorage
voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
models = voxel_storage.list_models()

print(f"Found {len(models)} models")
if models:
    model_id = models[0]
    print(f"Using model: {model_id}")
    
    # Get STL file path
    stl_path = stl_client.load_stl_file(model_id)
    if stl_path:
        print(f"✅ STL file found: {stl_path}")
    else:
        print("❌ STL file not found in database")
        stl_path = None
else:
    print("❌ No models found in database")
    stl_path = None
    model_id = None


TypeError: MongoDBClient.__init__() missing 1 required positional argument: 'config'

In [3]:
# Load STL file using different methods
stl_mesh = None
load_method = None

if stl_path and stl_path.exists():
    print(f"Loading STL from: {stl_path}")
    
    # Try PyVista first (best performance)
    if PYVISTA_AVAILABLE:
        try:
            stl_mesh = pv.read(str(stl_path))
            n_faces = stl_mesh.n_cells
            print(f"✅ PyVista loaded: {len(stl_mesh.points)} vertices, {n_faces} faces")
            load_method = 'pyvista'
        except Exception as e:
            print(f"⚠️ PyVista failed: {e}")
    
    # Try trimesh if PyVista failed
    if stl_mesh is None and TRIMESH_AVAILABLE:
        try:
            mesh_trimesh = trimesh.load(str(stl_path))
            if PYVISTA_AVAILABLE:
                # Convert to PyVista
                stl_mesh = pv.PolyData(mesh_trimesh.vertices, faces=mesh_trimesh.faces)
                print(f"✅ trimesh loaded and converted to PyVista: {len(mesh_trimesh.vertices)} vertices, {len(mesh_trimesh.faces)} faces")
                load_method = 'trimesh'
            else:
                stl_mesh = mesh_trimesh
                print(f"✅ trimesh loaded: {len(mesh_trimesh.vertices)} vertices, {len(mesh_trimesh.faces)} faces")
                load_method = 'trimesh'
        except Exception as e:
            print(f"⚠️ trimesh failed: {e}")
    
    # Try numpy-stl if both failed
    if stl_mesh is None and NUMPY_STL_AVAILABLE:
        try:
            stl_mesh = mesh.Mesh.from_file(str(stl_path))
            print(f"✅ numpy-stl loaded: {len(stl_mesh.vectors)} triangles")
            load_method = 'numpy-stl'
        except Exception as e:
            print(f"⚠️ numpy-stl failed: {e}")
    
    if stl_mesh is None:
        print("❌ Failed to load STL with any method")
else:
    print("❌ No STL path available")


NameError: name 'stl_path' is not defined

In [4]:
# Extract triangles from STL mesh with validation
def extract_triangles(mesh_obj, method, max_faces=2000):
    """Extract and validate triangles from STL mesh."""
    valid_triangles = []
    invalid_count = 0
    
    if method == 'pyvista':
        vertices = mesh_obj.points
        n_faces = mesh_obj.n_cells
        
        # Sample faces for performance
        if n_faces > max_faces:
            step = max(1, n_faces // max_faces)
            sampled_face_indices = list(range(0, n_faces, step))
        else:
            sampled_face_indices = list(range(n_faces))
        
        for face_idx in sampled_face_indices:
            try:
                cell = mesh_obj.get_cell(face_idx)
                if cell.n_points == 3:
                    point_ids = cell.point_ids
                    if len(point_ids) == 3:
                        triangle_verts = vertices[point_ids]
                        
                        # Validate triangle (check distances)
                        p0, p1, p2 = triangle_verts[0], triangle_verts[1], triangle_verts[2]
                        dist_01 = np.linalg.norm(p0 - p1)
                        dist_02 = np.linalg.norm(p0 - p2)
                        dist_12 = np.linalg.norm(p1 - p2)
                        
                        if dist_01 > 1e-6 and dist_02 > 1e-6 and dist_12 > 1e-6:
                            valid_triangles.append(triangle_verts)
                        else:
                            invalid_count += 1
            except Exception:
                invalid_count += 1
                continue
    
    elif method == 'trimesh':
        if PYVISTA_AVAILABLE and isinstance(mesh_obj, pv.PolyData):
            # Already converted to PyVista
            return extract_triangles(mesh_obj, 'pyvista', max_faces)
        else:
            vertices = mesh_obj.vertices
            faces = mesh_obj.faces
            
            # Sample faces for performance
            if len(faces) > max_faces:
                step = max(1, len(faces) // max_faces)
                sampled_faces = faces[::step]
            else:
                sampled_faces = faces
            
            for face in sampled_faces:
                if len(face) == 3:
                    triangle_verts = vertices[face]
                    
                    # Validate triangle
                    p0, p1, p2 = triangle_verts[0], triangle_verts[1], triangle_verts[2]
                    dist_01 = np.linalg.norm(p0 - p1)
                    dist_02 = np.linalg.norm(p0 - p2)
                    dist_12 = np.linalg.norm(p1 - p2)
                    
                    if dist_01 > 1e-6 and dist_02 > 1e-6 and dist_12 > 1e-6:
                        valid_triangles.append(triangle_verts)
                    else:
                        invalid_count += 1
    
    elif method == 'numpy-stl':
        vectors = mesh_obj.vectors
        
        # Sample for performance
        if len(vectors) > max_faces:
            step = max(1, len(vectors) // max_faces)
            sampled_vectors = vectors[::step]
        else:
            sampled_vectors = vectors
        
        for triangle in sampled_vectors:
            # Validate triangle
            p0, p1, p2 = triangle[0], triangle[1], triangle[2]
            dist_01 = np.linalg.norm(p0 - p1)
            dist_02 = np.linalg.norm(p0 - p2)
            dist_12 = np.linalg.norm(p1 - p2)
            
            if dist_01 > 1e-6 and dist_02 > 1e-6 and dist_12 > 1e-6:
                valid_triangles.append(triangle)
            else:
                invalid_count += 1
    
    if invalid_count > 0:
        print(f"⚠️ Skipped {invalid_count} invalid/degenerate triangles")
    
    return valid_triangles

if stl_mesh is not None:
    triangles = extract_triangles(stl_mesh, load_method, max_faces=2000)
    print(f"\n✅ Extracted {len(triangles)} valid triangles")
    if triangles:
        print(f"   First triangle shape: {triangles[0].shape}")
        print(f"   First triangle points:\n{triangles[0]}")
else:
    triangles = []
    print("❌ No mesh loaded")


❌ No mesh loaded


In [5]:
# Plot STL model with matplotlib - Filled mode using Poly3DCollection
if triangles:
    fig = plt.figure(figsize=(12, 5))
    
    # Filled rendering
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Use Poly3DCollection for filled triangles
    poly3d = Poly3DCollection(triangles, alpha=0.7, facecolor='lightblue', edgecolor='none')
    ax1.add_collection3d(poly3d)
    
    # Set axis limits based on triangle bounds
    all_points = np.vstack(triangles)
    ax1.set_xlim(all_points[:, 0].min(), all_points[:, 0].max())
    ax1.set_ylim(all_points[:, 1].min(), all_points[:, 1].max())
    ax1.set_zlim(all_points[:, 2].min(), all_points[:, 2].max())
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('Filled Rendering (Poly3DCollection)')
    
    # Wireframe rendering
    ax2 = fig.add_subplot(122, projection='3d')
    
    # Plot each triangle as wireframe
    for triangle in triangles[:500]:  # Limit for performance
        ax2.plot([triangle[0, 0], triangle[1, 0], triangle[2, 0], triangle[0, 0]],
                [triangle[0, 1], triangle[1, 1], triangle[2, 1], triangle[0, 1]],
                [triangle[0, 2], triangle[1, 2], triangle[2, 2], triangle[0, 2]],
                color='blue', alpha=0.5, linewidth=0.5)
    
    ax2.set_xlim(all_points[:, 0].min(), all_points[:, 0].max())
    ax2.set_ylim(all_points[:, 1].min(), all_points[:, 1].max())
    ax2.set_zlim(all_points[:, 2].min(), all_points[:, 2].max())
    
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')
    ax2.set_title('Wireframe Rendering')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ STL model rendered successfully!")
else:
    print("❌ No triangles to plot")


❌ No triangles to plot


In [6]:
# Test with different colors and opacity
if triangles:
    fig = plt.figure(figsize=(15, 5))
    
    colors = ['lightblue', 'lightcoral', 'lightgreen']
    alphas = [0.3, 0.6, 0.9]
    
    for i, (color, alpha) in enumerate(zip(colors, alphas)):
        ax = fig.add_subplot(131 + i, projection='3d')
        
        poly3d = Poly3DCollection(triangles, alpha=alpha, facecolor=color, edgecolor='none')
        ax.add_collection3d(poly3d)
        
        all_points = np.vstack(triangles)
        ax.set_xlim(all_points[:, 0].min(), all_points[:, 0].max())
        ax.set_ylim(all_points[:, 1].min(), all_points[:, 1].max())
        ax.set_zlim(all_points[:, 2].min(), all_points[:, 2].max())
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'Color: {color}, Alpha: {alpha}')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Tested different colors and opacity")


In [7]:
# Summary
print("\n" + "="*60)
print("STL Plotting Demo Summary")
print("="*60)
print(f"STL File: {stl_path}")
print(f"Load Method: {load_method}")
print(f"Valid Triangles: {len(triangles)}")
print(f"\n✅ All tests completed successfully!")
print("\nKey findings:")
print("- Poly3DCollection works well for filled triangle rendering")
print("- Triangle validation using distance checks prevents errors")
print("- PyVista is the preferred loading method")
print("- Sampling large meshes improves performance")



STL Plotting Demo Summary


NameError: name 'stl_path' is not defined